## Creates the doc2vec vector embeddings for a specific configuration

In [1]:
import json
import nltk
from nltk.tokenize import RegexpTokenizer
import string
import math
import os
import time
from collections import namedtuple
import cPickle as pickle
import pandas as pd

%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import random
import gzip

from multiprocessing import Process, Queue

from multiprocessing.dummy import Pool as ThreadPool
import itertools

from sklearn.metrics import coverage_error
import sklearn.metrics
from sklearn.multiclass import OneVsRestClassifier
from sklearn import linear_model
from sklearn.preprocessing import MultiLabelBinarizer

from gensim.models.doc2vec import Doc2Vec, LabeledSentence

import logging
from logging import info
from functools import partial

from thesis.utils.metrics import *
from thesis.utils.file import *

## Global variables used throughout the script

In [2]:
root = logging.getLogger()
for handler in root.handlers[:]:
    root.removeHandler(handler)
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO) # adds a default StreamHanlder
#root.addHandler(logging.StreamHandler())

In [3]:
SVM_SEED = 1234
DOC2VEC_SEED = 1234

In [4]:
MIN_WORD_COUNT = 100
NUM_CORES = 16

In [5]:
GLOBAL_VARS = namedtuple('GLOBAL_VARS', ['MODEL_NAME', 'DOC2VEC_MODEL_NAME', 'DOC2VEC_MODEL', 
                                         'SVM_MODEL_NAME', 'NN_MODEL_NAME'])

In [6]:
VOCAB_MODEL = "vocab_model"
MODEL_PREFIX = "model"
VALIDATION_DICT = "validation_dict.pkl"
TEST_MATRIX = "test_matrix.pkl"
TEST_DICT = "test_dict.pkl"
METRICS = "metrics.pkl"
CLASSIFIER = "classifier.pkl"

In [7]:
root_location = "/mnt/virtual-machines/data/"
exports_location = root_location + "exported_data/"

doc_classifications_map_file = exports_location + "doc_classification_map.pkl"
training_docs_list_file = exports_location + "training_docs_list.pkl"
validation_docs_list_file = exports_location + "validation_docs_list.pkl"
test_docs_list_file = exports_location + "test_docs_list.pkl"

preprocessed_location = root_location + "preprocessed_data/extended_pv_abs_desc_claims_full_chunks/"

training_preprocessed_files_prefix = preprocessed_location + "extended_pv_training_docs_data_preprocessed-"
validation_preprocessed_files_prefix = preprocessed_location + "extended_pv_validation_docs_data_preprocessed-"
test_preprocessed_files_prefix = preprocessed_location + "extended_pv_test_docs_data_preprocessed-"

## Load general data required for classification

In [8]:
%%time
doc_classification_map = pickle.load(open(doc_classifications_map_file))
training_docs_list = pickle.load(open(training_docs_list_file))
validation_docs_list = pickle.load(open(validation_docs_list_file))
test_docs_list = pickle.load(open(test_docs_list_file))

CPU times: user 21.7 s, sys: 1.47 s, total: 23.2 s
Wall time: 23.2 s


In [9]:
len(training_docs_list)

1286325

In [10]:
len(validation_docs_list)

321473

In [11]:
len(test_docs_list)

401877

# Utility functions for data loading

In [12]:
VALIDATION_MINI_BATCH_SIZE = 10000
def get_extended_docs_with_inference_data_only(doc2vec_model, file_to_write, preprocessed_files_prefix, level, model_name):
    """
    Use the trained doc2vec model to get the paragraph vector representations of the validation or test documents
    """

    def infer_one_doc(doc_tuple):
        # doc2vec_model.random = np.random.RandomState(DOC2VEC_SEED)
        doc_id, doc_tokens = doc_tuple
        rep = doc2vec_model.infer_vector(doc_tokens)
        return (doc_id, rep)

    if os.path.exists(os.path.join(doc2vec_model_save_location, GLOBAL_VARS.MODEL_NAME, file_to_write)):
        info("===== Loading inference vectors")
        inference_documents_reps = pickle.load(open(os.path.join(doc2vec_model_save_location, GLOBAL_VARS.MODEL_NAME, file_to_write)))
        info("Loaded inference vectors matrix")
    else:
        inference_documents_reps = {}
        info("===== Getting vectors with inference")

        # Multi-threaded inference
#         inference_docs_iterator = ExtendedPVDocumentBatchGenerator(preprocessed_files_prefix, batch_size=None)
        inference_docs_iterator = BatchWrapper(preprocessed_files_prefix, batch_size=None, level=level, level_type=model_name)
        generator_func = inference_docs_iterator.__iter__()
        # map consumes the whole iterator on the spot, so we have to use itertools.islice to fake mini-batching
        mini_batch_size = VALIDATION_MINI_BATCH_SIZE
        batches_run = 1
        pool = ThreadPool(NUM_CORES)
        while True:
            threaded_reps_partial = pool.map(infer_one_doc, itertools.islice(generator_func, mini_batch_size))
            info("Finished: {} tags".format(batches_run * mini_batch_size))
            batches_run += 1
            if threaded_reps_partial:
                # threaded_reps.extend(threaded_reps_partial)
                inference_documents_reps.update(threaded_reps_partial)
            else:
                break
                
        pool.close()
        pool.terminate()

        pickle.dump(inference_documents_reps,
                    open(os.path.join(doc2vec_model_save_location, GLOBAL_VARS.MODEL_NAME, file_to_write), 'w'))

    return inference_documents_reps

In [13]:
class ExtendedPVDocumentBatchGenerator(Process):
    def __init__(self, filename_prefix, queue, batch_size=10000, start_file=0, offset=10000):
        super(ExtendedPVDocumentBatchGenerator, self).__init__()
        self.queue = queue
        self.offset = offset
        self.filename_prefix = filename_prefix
        self.files_loaded = start_file - offset

    def run(self):
        cur_file = None
        while True:
            try:
                if cur_file is None:
                    info("Loading new file for index: {}".format(str(self.files_loaded + self.offset)))
                    cur_file = gzip.open(self.filename_prefix + str(self.files_loaded + self.offset) + '.gz')
#                     cur_file = open(self.filename_prefix + str(self.files_loaded + self.offset))
                    self.files_loaded += self.offset
                for line in cur_file:
                    self.queue.put(line)
                cur_file.close()
                cur_file = None
            except IOError:
                self.queue.put(False, block=True, timeout=None)
                info("All files are loaded - last file: {}".format(str(self.files_loaded + self.offset)))
                return


class BatchWrapper(object):
    def __init__(self, training_preprocessed_files_prefix, buffer_size=10000, batch_size=10000, level=1, level_type=None):
        assert batch_size <= 10000 or batch_size is None
        self.level = level
        self.level_type = level_type[0] if level_type is not None else None
        self.batch_size = batch_size
        self.q = Queue(maxsize=buffer_size)
        self.p = ExtendedPVDocumentBatchGenerator(training_preprocessed_files_prefix, queue=self.q,
                                                  batch_size=batch_size, start_file=0, offset=10000)
        self.p.start()
        self.cur_data = []

    def is_correct_type(self, doc_id):
        parts = doc_id.split("_")
        len_parts = len(parts)
        if len_parts == self.level:
            if len_parts == 1:
                return True
            if len_parts == self.level and (parts[1][0] == self.level_type or self.level_type is None):
                return True
        return False

    def return_sentences(self, line):
        line_array = tuple(line.split(" "))
        doc_id = line_array[0]
        if not self.is_correct_type(doc_id):
            return False
        line_array = line_array[1:]
        len_line_array = len(line_array)
        # divide the document to batches according to the batch size
        sentences = []
        
        if self.batch_size is None:
            # dont use LabeledSentence for validation iterator
            sentences.append((doc_id, line_array))
        else:
            curr_batch_iter = 0
            while curr_batch_iter < len_line_array:
                sentences.append(LabeledSentence(words=line_array[curr_batch_iter: curr_batch_iter + self.batch_size], tags=[doc_id]))
                curr_batch_iter += self.batch_size
        return tuple(sentences)

    def __iter__(self):
        while True:
            item = self.q.get(block=True)
            if item is False:
                raise StopIteration()
            else:
                sentences = self.return_sentences(item)
                if not sentences:
                    None
                else:
                    for sentence in sentences:
                        yield sentence


# Doc2vec and SVM Parameters

In [15]:
DOC2VEC_SIZE = 200
DOC2VEC_WINDOW = 2
DOC2VEC_MAX_VOCAB_SIZE = None
DOC2VEC_SAMPLE = 1e-3
DOC2VEC_TYPE = 1
DOC2VEC_HIERARCHICAL_SAMPLE = 0
DOC2VEC_NEGATIVE_SAMPLE_SIZE = 10
DOC2VEC_CONCAT = 0
DOC2VEC_MEAN = 1
DOC2VEC_TRAIN_WORDS = 0
DOC2VEC_EPOCHS = 1 # we do our training manually one epoch at a time
DOC2VEC_MAX_EPOCHS = 8
REPORT_DELAY = 20 # report the progress every x seconds
REPORT_VOCAB_PROGRESS = 100000 # report vocab progress every x documents

## Create the Doc2vec model and create/load the vocab

In [16]:
models = [
    (3, 'claims')
]
level, model_name = models[0]

In [24]:
info("creating/loading vocabulary for " + str(level) + ' ' + model_name + ' in ')
doc2vec_model_save_location = os.path.join(root_location,
                                           "parameter_search_doc2vec_models_" + str(level) + '_' + model_name,
                                           "full")
if not os.path.exists(doc2vec_model_save_location):
    os.makedirs(doc2vec_model_save_location)
if not os.path.exists(os.path.join(doc2vec_model_save_location, VOCAB_MODEL)):
    os.makedirs(os.path.join(doc2vec_model_save_location, VOCAB_MODEL))

placeholder_model_name = 'doc2vec_size_{}_w_{}_type_{}_concat_{}_mean_{}_trainwords_{}_hs_{}_neg_{}_vocabsize_{}_model_{}'.format(DOC2VEC_SIZE,
                                                                DOC2VEC_WINDOW,
                                                                'dm' if DOC2VEC_TYPE == 1 else 'pv-dbow',
                                                                DOC2VEC_CONCAT, DOC2VEC_MEAN,
                                                                DOC2VEC_TRAIN_WORDS,
                                                                DOC2VEC_HIERARCHICAL_SAMPLE,DOC2VEC_NEGATIVE_SAMPLE_SIZE,
                                                                str(DOC2VEC_MAX_VOCAB_SIZE),
                                                                str(level) + '_' + model_name
                                                                )
GLOBAL_VARS.DOC2VEC_MODEL_NAME = placeholder_model_name
placeholder_model_name = os.path.join(placeholder_model_name, "epoch_{}")
info("FILE " + os.path.join(doc2vec_model_save_location, VOCAB_MODEL, MODEL_PREFIX))
doc2vec_model = Doc2Vec(size=DOC2VEC_SIZE, window=DOC2VEC_WINDOW, min_count=MIN_WORD_COUNT,
                max_vocab_size= DOC2VEC_MAX_VOCAB_SIZE,
                sample=DOC2VEC_SAMPLE, seed=DOC2VEC_SEED, workers=NUM_CORES,
                # doc2vec algorithm dm=1 => PV-DM, dm=2 => PV-DBOW, PV-DM dictates CBOW for words
                dm=DOC2VEC_TYPE,
                # hs=0 => negative sampling, hs=1 => hierarchical softmax
                hs=DOC2VEC_HIERARCHICAL_SAMPLE, negative=DOC2VEC_NEGATIVE_SAMPLE_SIZE,
                dm_concat=DOC2VEC_CONCAT,
                # would train words with skip-gram on top of cbow, we don't need that for now
                dbow_words=DOC2VEC_TRAIN_WORDS,
                iter=DOC2VEC_EPOCHS)

GLOBAL_VARS.DOC2VEC_MODEL = doc2vec_model


2017-04-10 16:19:23,129 : INFO : creating/loading vocabulary for 1 document in 
2017-04-10 16:19:23,130 : INFO : FILE /mnt/virtual-machines/data/parameter_search_doc2vec_models_1_document/full/vocab_model/model


In [ ]:
if not os.path.exists(os.path.join(doc2vec_model_save_location, VOCAB_MODEL, MODEL_PREFIX)):
    
    info("Creating vocab model")
    training_docs_iterator = BatchWrapper(training_preprocessed_files_prefix, batch_size=10000, level=level,
                                          level_type=model_name)
    doc2vec_model.build_vocab(sentences=training_docs_iterator, progress_per=REPORT_VOCAB_PROGRESS)
    doc2vec_model.save(os.path.join(doc2vec_model_save_location, VOCAB_MODEL, MODEL_PREFIX))
else:
    info("Loading vocab model")
    doc2vec_model_vocab_model = Doc2Vec.load(os.path.join(doc2vec_model_save_location, VOCAB_MODEL, MODEL_PREFIX))
    doc2vec_model.reset_from(doc2vec_model_vocab_model)

## Actual Training, validation and Metrics Loop

In [25]:
doc2vec_model.min_alpha = 0.025
DOC2VEC_ALPHA_DECREASE = 0.001

In [26]:
doc2vec_model.workers = NUM_CORES

In [ ]:
%%time
# when resuming, resume from an epoch with a previously created doc2vec model to get the learning rate right
start_from = 1
for epoch in range(start_from, DOC2VEC_MAX_EPOCHS+1):
    GLOBAL_VARS.MODEL_NAME = placeholder_model_name.format(epoch)
    info("****************** Epoch {} --- Working on {} *******************".format(epoch, GLOBAL_VARS.MODEL_NAME))
    
    # if we have the model, just load it, otherwise train the previous model
    if os.path.exists(os.path.join(doc2vec_model_save_location, GLOBAL_VARS.MODEL_NAME, MODEL_PREFIX)):
        doc2vec_model = Doc2Vec.load(os.path.join(doc2vec_model_save_location, GLOBAL_VARS.MODEL_NAME, MODEL_PREFIX))
        doc2vec_model.workers = NUM_CORES
        GLOBAL_VARS.DOC2VEC_MODEL = doc2vec_model
    else:
        # train the doc2vec model
        training_docs_iterator = BatchWrapper(training_preprocessed_files_prefix, batch_size=10000, level=level,
                                          level_type=model_name)
        %time doc2vec_model.train(sentences=training_docs_iterator, report_delay=REPORT_DELAY)
        doc2vec_model.alpha -= DOC2VEC_ALPHA_DECREASE  # decrease the learning rate
        doc2vec_model.min_alpha = doc2vec_model.alpha  # fix the learning rate, no decay
        ensure_disk_location_exists(os.path.join(doc2vec_model_save_location, GLOBAL_VARS.MODEL_NAME))
        doc2vec_model.save(os.path.join(doc2vec_model_save_location, GLOBAL_VARS.MODEL_NAME, MODEL_PREFIX))
        GLOBAL_VARS.DOC2VEC_MODEL = doc2vec_model
        
    # only do the inference for higher epochs, as inference usually takes as much time as the actual training
    if epoch > 7:
        # Validation Embeddings
        info('Getting Validation Embeddings')
        Xv = get_extended_docs_with_inference_data_only(doc2vec_model, VALIDATION_DICT, 
                                         validation_preprocessed_files_prefix, level, model_name)

2017-04-09 16:56:40,606 : INFO : ****************** Epoch 1 --- Working on doc2vec_size_200_w_2_type_dm_concat_0_mean_1_trainwords_0_hs_0_neg_10_vocabsize_None_model_1_document/epoch_1 *******************
2017-04-09 16:56:40,735 : INFO : training model with 16 workers on 446814 vocabulary and 200 features, using sg=0 hs=0 sample=0.001 negative=10 window=2
2017-04-09 16:56:40,735 : INFO : Loading new file for index: 0
2017-04-09 16:56:40,736 : INFO : expecting 1879865 sentences, matching count from corpus used for vocabulary survey
2017-04-09 16:56:41,782 : INFO : PROGRESS: at 0.01% examples, 610582 words/s, in_qsize 0, out_qsize 2
2017-04-09 16:57:01,783 : INFO : PROGRESS: at 0.20% examples, 837420 words/s, in_qsize 0, out_qsize 2
2017-04-09 16:57:21,803 : INFO : PROGRESS: at 0.41% examples, 854427 words/s, in_qsize 0, out_qsize 2
2017-04-09 16:57:41,806 : INFO : PROGRESS: at 0.61% examples, 854564 words/s, in_qsize 0, out_qsize 0
2017-04-09 16:57:55,704 : INFO : Loading new file for i

2017-04-09 17:18:22,190 : INFO : PROGRESS: at 12.73% examples, 842020 words/s, in_qsize 0, out_qsize 0
2017-04-09 17:18:42,196 : INFO : PROGRESS: at 12.92% examples, 841905 words/s, in_qsize 0, out_qsize 2
2017-04-09 17:19:02,200 : INFO : PROGRESS: at 13.12% examples, 841955 words/s, in_qsize 0, out_qsize 0
2017-04-09 17:19:15,216 : INFO : Loading new file for index: 170000
2017-04-09 17:19:22,211 : INFO : PROGRESS: at 13.32% examples, 841940 words/s, in_qsize 0, out_qsize 0
2017-04-09 17:19:42,221 : INFO : PROGRESS: at 13.51% examples, 841777 words/s, in_qsize 0, out_qsize 2
2017-04-09 17:20:02,225 : INFO : PROGRESS: at 13.71% examples, 841798 words/s, in_qsize 0, out_qsize 0
2017-04-09 17:20:22,230 : INFO : PROGRESS: at 13.90% examples, 841741 words/s, in_qsize 0, out_qsize 0
2017-04-09 17:20:36,235 : INFO : Loading new file for index: 180000
2017-04-09 17:20:42,233 : INFO : PROGRESS: at 14.09% examples, 841749 words/s, in_qsize 0, out_qsize 0
2017-04-09 17:21:02,234 : INFO : PROGRES

2017-04-09 17:41:22,701 : INFO : PROGRESS: at 26.34% examples, 845249 words/s, in_qsize 0, out_qsize 0
2017-04-09 17:41:36,462 : INFO : Loading new file for index: 340000
2017-04-09 17:41:42,705 : INFO : PROGRESS: at 26.54% examples, 845396 words/s, in_qsize 0, out_qsize 0
2017-04-09 17:42:02,710 : INFO : PROGRESS: at 26.73% examples, 845352 words/s, in_qsize 0, out_qsize 0
2017-04-09 17:42:22,711 : INFO : PROGRESS: at 26.93% examples, 845511 words/s, in_qsize 0, out_qsize 0
2017-04-09 17:42:42,717 : INFO : PROGRESS: at 27.13% examples, 845558 words/s, in_qsize 0, out_qsize 1
2017-04-09 17:42:53,797 : INFO : Loading new file for index: 350000
2017-04-09 17:43:02,725 : INFO : PROGRESS: at 27.33% examples, 845620 words/s, in_qsize 0, out_qsize 0
2017-04-09 17:43:22,732 : INFO : PROGRESS: at 27.53% examples, 845672 words/s, in_qsize 0, out_qsize 0
2017-04-09 17:43:42,737 : INFO : PROGRESS: at 27.72% examples, 845657 words/s, in_qsize 0, out_qsize 1
2017-04-09 17:44:02,743 : INFO : PROGRES

2017-04-09 18:04:03,170 : INFO : PROGRESS: at 39.85% examples, 848209 words/s, in_qsize 0, out_qsize 0
2017-04-09 18:04:23,171 : INFO : PROGRESS: at 40.05% examples, 848206 words/s, in_qsize 0, out_qsize 0
2017-04-09 18:04:43,171 : INFO : PROGRESS: at 40.25% examples, 848192 words/s, in_qsize 0, out_qsize 0
2017-04-09 18:05:02,526 : INFO : Loading new file for index: 520000
2017-04-09 18:05:03,185 : INFO : PROGRESS: at 40.45% examples, 848229 words/s, in_qsize 0, out_qsize 1
2017-04-09 18:05:23,191 : INFO : PROGRESS: at 40.65% examples, 848261 words/s, in_qsize 0, out_qsize 1
2017-04-09 18:05:43,192 : INFO : PROGRESS: at 40.83% examples, 848040 words/s, in_qsize 0, out_qsize 3
2017-04-09 18:06:03,196 : INFO : PROGRESS: at 41.04% examples, 848079 words/s, in_qsize 0, out_qsize 1
2017-04-09 18:06:20,747 : INFO : Loading new file for index: 530000
2017-04-09 18:06:23,204 : INFO : PROGRESS: at 41.24% examples, 848103 words/s, in_qsize 0, out_qsize 2
2017-04-09 18:06:43,207 : INFO : PROGRES

2017-04-09 18:27:03,636 : INFO : PROGRESS: at 53.55% examples, 849300 words/s, in_qsize 0, out_qsize 0
2017-04-09 18:27:11,610 : INFO : Loading new file for index: 690000
2017-04-09 18:27:23,636 : INFO : PROGRESS: at 53.75% examples, 849329 words/s, in_qsize 0, out_qsize 0
2017-04-09 18:27:43,645 : INFO : PROGRESS: at 53.94% examples, 849320 words/s, in_qsize 0, out_qsize 0
2017-04-09 18:28:03,646 : INFO : PROGRESS: at 54.14% examples, 849336 words/s, in_qsize 0, out_qsize 0
2017-04-09 18:28:23,648 : INFO : PROGRESS: at 54.34% examples, 849356 words/s, in_qsize 0, out_qsize 0
2017-04-09 18:28:29,907 : INFO : Loading new file for index: 700000
2017-04-09 18:28:43,657 : INFO : PROGRESS: at 54.54% examples, 849370 words/s, in_qsize 0, out_qsize 1
2017-04-09 18:29:03,664 : INFO : PROGRESS: at 54.74% examples, 849397 words/s, in_qsize 0, out_qsize 0
2017-04-09 18:29:23,676 : INFO : PROGRESS: at 54.93% examples, 849340 words/s, in_qsize 0, out_qsize 0
2017-04-09 18:29:43,681 : INFO : PROGRES

...

2017-04-10 14:44:18,308 : INFO : PROGRESS: at 78.56% examples, 842015 words/s, in_qsize 0, out_qsize 2
2017-04-10 14:44:38,312 : INFO : PROGRESS: at 78.76% examples, 842078 words/s, in_qsize 0, out_qsize 1
2017-04-10 14:44:58,323 : INFO : PROGRESS: at 78.95% examples, 842114 words/s, in_qsize 0, out_qsize 6
2017-04-10 14:45:18,325 : INFO : PROGRESS: at 79.15% examples, 842127 words/s, in_qsize 0, out_qsize 2
2017-04-10 14:45:36,702 : INFO : Loading new file for index: 1020000
2017-04-10 14:45:38,338 : INFO : PROGRESS: at 79.35% examples, 842116 words/s, in_qsize 0, out_qsize 1
2017-04-10 14:45:58,352 : INFO : PROGRESS: at 79.54% examples, 842095 words/s, in_qsize 0, out_qsize 0
2017-04-10 14:46:18,358 : INFO : PROGRESS: at 79.74% examples, 842080 words/s, in_qsize 0, out_qsize 0
2017-04-10 14:46:38,363 : INFO : PROGRESS: at 79.93% examples, 842079 words/s, in_qsize 1, out_qsize 1
2017-04-10 14:46:57,276 : INFO : Loading new file for index: 1030000
2017-04-10 14:46:58,370 : INFO : PROGR

2017-04-10 15:07:18,740 : INFO : PROGRESS: at 92.33% examples, 844335 words/s, in_qsize 0, out_qsize 0
2017-04-10 15:07:38,739 : INFO : PROGRESS: at 92.54% examples, 844391 words/s, in_qsize 0, out_qsize 2
2017-04-10 15:07:39,693 : INFO : Loading new file for index: 1190000
2017-04-10 15:07:58,739 : INFO : PROGRESS: at 92.74% examples, 844432 words/s, in_qsize 0, out_qsize 1
2017-04-10 15:08:18,746 : INFO : PROGRESS: at 92.94% examples, 844488 words/s, in_qsize 1, out_qsize 0
2017-04-10 15:08:38,753 : INFO : PROGRESS: at 93.14% examples, 844527 words/s, in_qsize 0, out_qsize 0
2017-04-10 15:08:55,650 : INFO : Loading new file for index: 1200000
2017-04-10 15:08:58,757 : INFO : PROGRESS: at 93.34% examples, 844565 words/s, in_qsize 0, out_qsize 0
2017-04-10 15:09:18,768 : INFO : PROGRESS: at 93.54% examples, 844602 words/s, in_qsize 0, out_qsize 0
2017-04-10 15:09:38,776 : INFO : PROGRESS: at 93.75% examples, 844629 words/s, in_qsize 0, out_qsize 0
2017-04-10 15:09:58,775 : INFO : PROGR

CPU times: user 20h 2min 4s, sys: 13min 12s, total: 20h 15min 17s
Wall time: 2h 49min 21s


2017-04-10 15:19:56,693 : INFO : not storing attribute syn0norm
2017-04-10 15:19:56,694 : INFO : storing np array 'syn0' to /mnt/virtual-machines/data/parameter_search_doc2vec_models_1_document/full/doc2vec_size_200_w_2_type_dm_concat_0_mean_1_trainwords_0_hs_0_neg_10_vocabsize_None_model_1_document/epoch_8/model.wv.syn0.npy
2017-04-10 15:19:56,930 : INFO : storing np array 'syn1neg' to /mnt/virtual-machines/data/parameter_search_doc2vec_models_1_document/full/doc2vec_size_200_w_2_type_dm_concat_0_mean_1_trainwords_0_hs_0_neg_10_vocabsize_None_model_1_document/epoch_8/model.syn1neg.npy
2017-04-10 15:19:57,131 : INFO : not storing attribute cum_table
2017-04-10 15:20:12,114 : INFO : saved /mnt/virtual-machines/data/parameter_search_doc2vec_models_1_document/full/doc2vec_size_200_w_2_type_dm_concat_0_mean_1_trainwords_0_hs_0_neg_10_vocabsize_None_model_1_document/epoch_8/model
2017-04-10 15:20:12,115 : INFO : Getting Validation Embeddings
2017-04-10 15:20:12,115 : INFO : ===== Getting ve

NameError: global name 'BatchClass' is not defined

## Inference Only (if needed)

In [50]:
NUM_CORES = 32

In [51]:
epoch = 8
GLOBAL_VARS.MODEL_NAME = placeholder_model_name.format(epoch)

if os.path.exists(os.path.join(doc2vec_model_save_location, GLOBAL_VARS.MODEL_NAME, MODEL_PREFIX)):
    doc2vec_model = Doc2Vec.load(os.path.join(doc2vec_model_save_location, GLOBAL_VARS.MODEL_NAME, MODEL_PREFIX))
    doc2vec_model.workers = NUM_CORES
    GLOBAL_VARS.DOC2VEC_MODEL = doc2vec_model

2017-04-10 17:03:27,806 : INFO : loading Doc2Vec object from /mnt/virtual-machines/data/parameter_search_doc2vec_models_1_document/full/doc2vec_size_200_w_2_type_dm_concat_0_mean_1_trainwords_0_hs_0_neg_10_vocabsize_None_model_1_document/epoch_8/model
2017-04-10 17:03:36,871 : INFO : loading docvecs recursively from /mnt/virtual-machines/data/parameter_search_doc2vec_models_1_document/full/doc2vec_size_200_w_2_type_dm_concat_0_mean_1_trainwords_0_hs_0_neg_10_vocabsize_None_model_1_document/epoch_8/model.docvecs.* with mmap=None
2017-04-10 17:03:36,872 : INFO : loading doctag_syn0 from /mnt/virtual-machines/data/parameter_search_doc2vec_models_1_document/full/doc2vec_size_200_w_2_type_dm_concat_0_mean_1_trainwords_0_hs_0_neg_10_vocabsize_None_model_1_document/epoch_8/model.docvecs.doctag_syn0.npy with mmap=None
2017-04-10 17:03:37,677 : INFO : loading wv recursively from /mnt/virtual-machines/data/parameter_search_doc2vec_models_1_document/full/doc2vec_size_200_w_2_type_dm_concat_0_mean

In [52]:
doc2vec_model

In [ ]:
Xv = get_extended_docs_with_inference_data_only(doc2vec_model, VALIDATION_DICT, 
                                         validation_preprocessed_files_prefix, level, model_name)

2017-04-10 17:12:48,776 : INFO : ===== Getting vectors with inference
2017-04-10 17:12:49,061 : INFO : Loading new file for index: 0
2017-04-10 17:13:11,811 : INFO : Finished: 1000 tags
2017-04-10 17:13:33,430 : INFO : Finished: 2000 tags
2017-04-10 17:13:56,085 : INFO : Finished: 3000 tags
2017-04-10 17:14:17,651 : INFO : Finished: 4000 tags
2017-04-10 17:14:39,593 : INFO : Finished: 5000 tags
2017-04-10 17:15:02,056 : INFO : Finished: 6000 tags
2017-04-10 17:15:24,133 : INFO : Finished: 7000 tags
2017-04-10 17:15:45,809 : INFO : Finished: 8000 tags
2017-04-10 17:16:07,193 : INFO : Finished: 9000 tags
2017-04-10 17:16:08,702 : INFO : Loading new file for index: 10000
2017-04-10 17:16:29,323 : INFO : Finished: 10000 tags
2017-04-10 17:16:51,594 : INFO : Finished: 11000 tags
2017-04-10 17:17:13,626 : INFO : Finished: 12000 tags
2017-04-10 17:17:36,071 : INFO : Finished: 13000 tags
2017-04-10 17:17:58,521 : INFO : Finished: 14000 tags
2017-04-10 17:18:20,964 : INFO : Finished: 15000 tags

2017-04-10 18:02:02,467 : INFO : Finished: 134000 tags
2017-04-10 18:02:23,534 : INFO : Finished: 135000 tags
2017-04-10 18:02:45,161 : INFO : Finished: 136000 tags
2017-04-10 18:03:06,566 : INFO : Finished: 137000 tags
2017-04-10 18:03:28,294 : INFO : Finished: 138000 tags
2017-04-10 18:03:49,788 : INFO : Finished: 139000 tags
2017-04-10 18:03:51,478 : INFO : Loading new file for index: 140000
2017-04-10 18:04:11,539 : INFO : Finished: 140000 tags
2017-04-10 18:04:32,958 : INFO : Finished: 141000 tags
2017-04-10 18:04:54,908 : INFO : Finished: 142000 tags
2017-04-10 18:05:16,756 : INFO : Finished: 143000 tags
2017-04-10 18:05:39,417 : INFO : Finished: 144000 tags
2017-04-10 18:06:01,588 : INFO : Finished: 145000 tags
2017-04-10 18:06:23,596 : INFO : Finished: 146000 tags
2017-04-10 18:06:45,153 : INFO : Finished: 147000 tags
2017-04-10 18:07:07,241 : INFO : Finished: 148000 tags
2017-04-10 18:07:29,414 : INFO : Finished: 149000 tags
2017-04-10 18:07:30,810 : INFO : Loading new file fo

2017-04-10 18:56:07,656 : INFO : Finished: 282000 tags
2017-04-10 18:56:29,354 : INFO : Finished: 283000 tags
2017-04-10 18:56:50,790 : INFO : Finished: 284000 tags
2017-04-10 18:57:13,031 : INFO : Finished: 285000 tags
2017-04-10 18:57:35,284 : INFO : Finished: 286000 tags
2017-04-10 18:57:57,129 : INFO : Finished: 287000 tags
2017-04-10 18:58:19,175 : INFO : Finished: 288000 tags
2017-04-10 18:58:41,498 : INFO : Finished: 289000 tags
2017-04-10 18:58:43,084 : INFO : Loading new file for index: 290000
2017-04-10 18:59:03,646 : INFO : Finished: 290000 tags
2017-04-10 18:59:25,522 : INFO : Finished: 291000 tags
2017-04-10 18:59:47,465 : INFO : Finished: 292000 tags
2017-04-10 19:00:08,736 : INFO : Finished: 293000 tags
2017-04-10 19:00:30,792 : INFO : Finished: 294000 tags
2017-04-10 19:00:52,469 : INFO : Finished: 295000 tags
2017-04-10 19:01:15,068 : INFO : Finished: 296000 tags
2017-04-10 19:01:36,886 : INFO : Finished: 297000 tags
2017-04-10 19:01:58,471 : INFO : Finished: 298000 ta

#### Testing inference

In [55]:
inference_docs_iterator = BatchWrapper(validation_preprocessed_files_prefix, batch_size=None, level=level, level_type=model_name)        
for doc_tuple in inference_doczs_iterator:
    doc_id, doc_tokens = doc_tuple
    rep = doc2vec_model.infer_vector(doc_tokens)
    print (doc_id, rep)
    break

('08521002', array([  1.13558674e+00,  -2.01971769e-01,  -9.30447519e-01,
         9.55632687e-01,   5.11517346e-01,   4.34441900e+00,
        -3.77764761e-01,  -1.11617422e+00,  -2.15896085e-01,
         9.09354746e-01,   5.74674904e-01,  -2.07049704e+00,
        -7.20400810e-01,   4.94136661e-01,  -1.74060893e+00,
        -2.17272949e+00,  -4.39270258e-01,  -1.51936769e+00,
         5.65607429e-01,  -4.58835810e-01,  -1.69598356e-01,
         1.77733886e+00,   3.66123140e-01,   1.38953611e-01,
        -1.04259264e+00,   8.84979665e-01,  -8.56729895e-02,
        -6.04329109e-01,   4.42179322e-01,   1.08561194e+00,
        -2.49654725e-01,   3.02951038e-01,  -3.80307257e-01,
         1.32433748e+00,   7.18038738e-01,   7.99864233e-01,
        -3.60305488e-01,  -3.32749695e-01,   1.86409019e-02,
        -1.16298962e+00,  -2.36521304e-01,   8.52507114e-01,
        -4.25269688e-03,  -2.73190904e+00,  -1.39228487e+00,
         3.14658254e-01,   1.19927609e+00,  -8.86219382e-01,
        -2.

In [53]:
doc2vec_model.wv.syn0

array([[ 0.24962339,  0.03808838, -0.38492572, ...,  0.81019139,
        -0.0872335 ,  0.00503489],
       [-0.24393913, -0.9072656 , -0.08245134, ..., -0.12438237,
        -0.10501056,  0.07241193],
       [ 0.06769085, -0.22004843,  0.05649997, ...,  0.15331532,
        -0.87121236, -0.71148068],
       ..., 
       [ 0.02257917,  0.18380728, -0.19475998, ...,  0.72972393,
        -0.03356596, -0.29145467],
       [-0.20255305, -0.25994578,  0.31640032, ..., -0.02623975,
         0.41660461, -0.45980361],
       [ 1.09419656, -0.97489876, -0.3509953 , ...,  0.82430571,
         0.02756385,  0.7905944 ]], dtype=float32)